# Book: C# 9.0 in a Nutshell 

## Chapter 4 : Advanced C#

- Delegates
- Events
- Lambda Expressions
- Anonymous Methods
- try Statements and Exceptions
- Enumerations and Iterators
- Nullable Value Types
- Nullable Reference Types
- Extension Methods
- Anonymous Types
- Tuples
- Records
- Patterns
- Attributes
- Caller Info Attributes
- Dynamic Binding
- Operator Overloading
- Unsafe code and Pointers
- Preprocessor Directives
- XML Documentation


### Delegates

- https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/delegates/
- Kind of like a function pointer but more than a function pointer
- A delegate is a type that represents references to methods with a particular parameter list and return type.
- Good for callback methods
- Delegates can be chained together
- 



In [1]:
// first write the method signature
// public int PerformCalculations(int x, int y);
// now add the delegate word 

public delegate int DoMath(int x, int y);

public int Add(int x, int y) => x + y;
public int Sub(int x, int y) => x - y;

DoMath dAdd = Add;
DoMath dSub = Sub;

Console.WriteLine(dAdd(2,3));
Console.WriteLine(dSub.Invoke(2,4));

In [1]:
// Can pass delegates as function parameter

public int WhatKindOfMathDoIPerform(int x, int y, DoMath dm)
{
    return dm(x,y);
}

Console.WriteLine(WhatKindOfMathDoIPerform(2, 3, dAdd));
Console.WriteLine(WhatKindOfMathDoIPerform(2, 3, dSub));


In [1]:
// https://stackoverflow.com/questions/2253874/standard-deviation-in-linq
// This uses Welford's method which has higher numerical accuracy 
// compared to the Average(x^2)-Average(x)^2 method.
public static double StdDev(IEnumerable<double> values)
{
    // ref: http://warrenseen.com/blog/2006/03/13/how-to-calculate-standard-deviation/
    double mean = 0.0;
    double sum = 0.0;
    double stdDev = 0.0;
    int n = 0;
    foreach (double val in values)
    {
        n++;
        double delta = val - mean;
        mean += delta / n;
        sum += delta * (val - mean);
    }
    if (1 < n)
        stdDev = Math.Sqrt(sum / (n - 1));

    return stdDev;
}



In [1]:

// run many operations at the same time
// ! beware : return types are lost except for the last one
// * there is a solution : GetInvocationList
// https://docs.microsoft.com/en-us/previous-versions/visualstudio/visual-studio-2008/ff518994(v=orm.10)
// step through the list and invoke each method manually
// hence using void 
public delegate void PrintStats(double[] x);

public void PrintMax(double[] x) => Console.WriteLine($"max: {x.Max()}");
public void PrintMin(double[] x) => Console.WriteLine($"min: {x.Min()}");
public void PrintAvg(double[] x) => Console.WriteLine($"avg: {x.Average()}");
public void PrintCount(double[] x) => Console.WriteLine($"count: {x.Count()}");
public void PrintSum(double[] x) => Console.WriteLine($"sum: {x.Sum()}");
// ! linq doesn't provide std dev : look at method def in the previous code cell
public void PrintStdDev(double[] x) => Console.WriteLine($"std dev: {StdDev(x)}");

PrintStats stats = Max;
stats += PrintMin;
stats += PrintAvg;
stats += PrintCount;
stats += PrintSum;
stats += PrintStdDev;

stats(new double[]{1,2,3,4,5});



In [1]:
// hmmm...so we can run many ops 
public delegate void LoggedIn(string loginName);

// let me know when someone logs in, so i can write to log
public void WriteToLog(string loginName) 
            => Console.WriteLine($"{loginName} logged in at {DateTime.Now}");

// let me know when someone logs in, so i can update dashboard 
public static int activeUsers = 0;
public void BumpActiveUser(string loginName) 
            => Console.WriteLine($"Active Users: {++activeUsers}");

// logged in user delegate
LoggedIn dLgdIn = WriteToLog;
dLgdIn += BumpActiveUser;

// null will raise exception
//dLgdIn = null;

// login method
public bool Login(string loginName, string password) 
{
    bool authenticated = false;
    authenticated =  
                // bart simpson from 'Simpsons'
                (loginName == "bart") && (password == "Eat my shorts")
                // fry from 'Futurama'
                || (loginName == "fry") && (password == "Shut up and take my money!");
    
    // notify listeners
    //if(authenticated) dLgdIn(loginName);
    if(authenticated) dLgdIn?.Invoke(loginName);

    return authenticated;
}


In [1]:

// let's log in and test out the above code
Login("bart", "Eat my shorts");
Login("homer", "D'oh!");
Login("fry", "Shut up and take my money!");
Login("bender", "Bite my Shiny Metal Ass");


### Generic Delegates



In [1]:
public delegate void PrintStats<T>(T[] arg);

PrintStats<double> dPS_G = PrintMin;
dPS_G(new double[]{22, 45, 6});

#### Funcs and Action Delegates

- Funcs : If they return a value
- Action : If they don't return a value
- https://docs.microsoft.com/en-us/dotnet/api/system.func-1?view=net-5.0
- https://docs.microsoft.com/en-us/dotnet/api/system.action?view=net-5.0

delegate TResult Func <out TResult>                ();  
delegate TResult Func <in T, out TResult>          (T arg);  
delegate TResult Func <in T1, in T2, out TResult>  (T1 arg1, T2 arg2);  
... and so on, up to T16  

delegate void Action                 ();  
delegate void Action <in T>          (T arg);  
delegate void Action <in T1, in T2>  (T1 arg1, T2 arg2);  
... and so on, up to T16  

In [1]:
// Func<double> dPS_f = PrintMin;
// Func<double []> dPS_fa = PrintMin;
// Action<double> dPS_a = PrintMin;

Action<double []> dPS_aa = PrintMin;
dPS_aa(new double[]{22, 45, 6});


Events

In [1]:
public delegate void PriceChangedHandler (decimal oldPrice,decimal newPrice);

public class Stock
{
    string symbol;
    decimal price;

    public Stock (string symbol) => this.symbol = symbol;

    public event PriceChangedHandler PriceChanged;

    public decimal Price
    {
        get => price;
        set
        {
            if (price == value) return;      // Exit if nothing has changed

            decimal oldPrice = price;
            price = value;
            
            if (PriceChanged != null)           // If invocation list not
                PriceChanged (oldPrice, price);   // empty, fire event.
        }
    }
}

Standard Event Pattern

In [1]:
public class PriceChangedEventArgs : System.EventArgs
{
    public readonly decimal LastPrice;
    public readonly decimal NewPrice;

    public PriceChangedEventArgs (decimal lastPrice, decimal newPrice)
    {
        LastPrice = lastPrice;
        NewPrice = newPrice;
    }
}

In [1]:
public class Stokk
{
    decimal price;

    public Stokk (string symbol) => Symbol = symbol;

    public event EventHandler<PriceChangedEventArgs> PriceChanged;

    protected virtual void OnPriceChanged (PriceChangedEventArgs e)
    {
        PriceChanged?.Invoke(this, e);
    }

    public string Symbol { get; init; }

    public decimal Price
    {
        get => price;
        set
        {
            if (price == value) return;      // Exit if nothing has changed

            decimal oldPrice = price;
            price = value;
            
            OnPriceChanged (new PriceChangedEventArgs(oldPrice, price));   // empty, fire event.
        }
    }
}

In [1]:
void stokk_PriceHasChanged(object sender, PriceChangedEventArgs e)
{
    if(e.NewPrice > e.LastPrice)
        { Console.WriteLine($"{(sender as Stokk).Symbol} rose from {e.LastPrice} to {e.NewPrice}"); }
    else if( e.NewPrice < e.LastPrice )
        { Console.WriteLine($"{(sender as Stokk).Symbol} fell from {e.LastPrice} to {e.NewPrice}"); }
    else 
        { Console.WriteLine($"{(sender as Stokk).Symbol} didn't change in price"); }
}

Stokk apple = new Stokk("APPL");
apple.Price = 40.0M;
apple.PriceChanged += stokk_PriceHasChanged;
// price rose
apple.Price = 4000.0M;
// price fell
apple.Price = 3900.0M;
// price stayed same
apple.Price = 3900.0M;


In [1]:
// sync vs async delegate calls
// can we delay the execution
using System.Threading;

Action<int> TypeThisEssay;

// fast 
public void ICanType_100_Words_a_Sec(int words) => ICanType_X_Words_a_Sec(100, words);

// slow
public void ICanType_20_Words_a_Sec(int words) => ICanType_X_Words_a_Sec(20,words);

public void ICanType_X_Words_a_Sec(int speed, int words)
{
    double time = (words*1.0/speed) * 1000; 
    Thread.Sleep((int)time); 
    Console.WriteLine($"{speed} words/sec time taken: {time} msec"); 
}

TypeThisEssay = ICanType_100_Words_a_Sec;
TypeThisEssay += ICanType_20_Words_a_Sec;

// sync call
int shortEssay = 45;
TypeThisEssay?.Invoke(shortEssay);
Console.WriteLine("All writers finished writing.");


In [1]:
// Asyncrhonous programming model
// https://docs.microsoft.com/en-us/dotnet/standard/asynchronous-programming-patterns/asynchronous-programming-model-apm
// The callback that gets called when 
// is finished processing

// later on we will discuss async await delegates

// for AsyncResult
using System.Runtime.Remoting.Messaging; 

// AsyncResult doesn't exist in .NET Core
private static void EssayWritten(IAsyncResult iResult)
{
    AsyncResult result = (AsyncResult)iResult;

    Action performQuickly = (Action)result.AsyncDelegate;

    performQuickly.EndInvoke(result);

    Console.WriteLine("All writers finished writing.");
}

AsyncCallback cbEssayWritten = new AsyncCallback(EssayWritten);
TypeThisEssay?.BeginInvoke(100, cbEssayWritten, null);

Console.WriteLine("Let's not wait for all writers to finish! Carry On!");



In [1]:
// Task Based asynchronous pattern
// https://docs.microsoft.com/en-us/dotnet/standard/asynchronous-programming-patterns/task-based-asynchronous-pattern-tap
// https://devblogs.microsoft.com/dotnet/migrating-delegate-begininvoke-calls-for-net-core/
// https://blog.stephencleary.com/2014/02/synchronous-and-asynchronous-delegate.html


Task tskTypeThisEssay = Task.Run( () => TypeThisEssay?.Invoke(shortEssay) );
Console.WriteLine("Let's not wait for all writers to finish! Carry On!");

// super fast
public void ICanType_200_Words_a_Sec(int words) => ICanType_X_Words_a_Sec(200,words);

ICanType_200_Words_a_Sec(shortEssay);

// what happens if we comment the following line
// we don't await
await tskTypeThisEssay;
Console.WriteLine("All writers finished writing.");
